In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import copy
import plotly.express as px
import plotly.graph_objects as go
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable = True, theme = 'pearl', offline = True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('..//input/how-to-do-product-analytics/product.csv')

In [ ]:
data.head()

In [ ]:
data.info()

### The dataframe has too high memory usage, so here I get more detailed information about it:

In [ ]:
data.info(memory_usage = 'deep')

### Dataframe uses 4.3 GB of memory, so the first step to decrease this value is deletting columns which I will not use. These are order_id and page_id.

In [ ]:
data = data.drop(['order_id', 'page_id'], axis = 1)
data.info(memory_usage = 'deep')

### Also there is a target column in the dataframe and I need to understand what information it gives.

In [ ]:
print('target column unique values: ', data.target.unique())
print('-----------------------------------------------------')
print('target column in the dataframe = 0')
print(data[data.target == 0])
print('-----------------------------------------------------')
print('target column in the dataframe = 1')
print(data[data.target == 1])

### It looks like target is 0 for banner_show and banner_click and target is 1 for order in title columns. Checking hypothesis.

In [ ]:
print('Is there target is 1 for banner_show and banner_click in title column')
if data[(data.target == 1)&((data.title == 'banner_show')|(data.title == 'banner_click'))].empty == True:
    print('There are not such rows in the dataframe')
else:
    data[(data.target == 1)&((data.title == 'banner_show')|(data.title == 'banner_click'))]
print('Is there target is 0 for order in title column')
if data[(data.target == 0)&(data.title == 'order')].empty == True:
    print('There are not such rows in the dataframe')
else:
    data[(data.target == 0)&(data.title == 'order')]

### My hypothesis was right: target column shows a good is ordered or not. We can get this information from the title column ('order' value), so I can delete this column too.

In [ ]:
data = data.drop('target', axis = 1)
data.info(memory_usage = 'deep')

### I deleted all unuseful columns, so now I should to manipulate data types in dataframe.

### We can change object types to category if there are less than 50% percent of unique values. Here I see how much unique values has every column.

In [ ]:
data.select_dtypes('object').nunique()

### The product, site_version and title columns have only 5, 2 and 3 unique values, so it is a good opportunity to change them to the category type.

In [ ]:
for col in ['product', 'site_version', 'title']:
    data[col] = data[col].astype('category')
print(data.info(memory_usage = 'deep'))
data.head()

In [ ]:
for dtype in ['int','object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [ ]:
data['time'] = pd.to_datetime(data['time'])
data['date'] = data['time'].dt.date
data['hour'] = data['time'].dt.hour
data = data.drop('time', axis = 1)
data['hour'] = pd.to_numeric(data['hour'])

In [ ]:
data.info()

### Check missing values

In [ ]:
data.isnull().sum()

#### Now I have a dataframe with 8 471 220 rows and 6 columns and I am ready to analyse it.

In [ ]:
data.shape

#### Making a copy of the dataframe

In [ ]:
data2 = copy.deepcopy(data)

#### See what products, site versions and page events we have

In [ ]:
print('Products on banners: ', data2['product'].unique())
print('Site versions: ', data2.site_version.unique())
print('Page events: ', data2.title.unique())

#### See site funnel. It has 17% conversion from show to click, 32% conversion from click to order and 5.6% conversion from show to order.

In [ ]:
fig = go.Figure(go.Funnel(x = data2.groupby('title').user_id.nunique().reset_index().sort_values('user_id', ascending = False)['user_id'],
                          y = data2.groupby('title').user_id.nunique().reset_index().sort_values('user_id', ascending = False)['title'],
                           textinfo = "value+percent initial"))
fig.show()

In [ ]:
px.funnel(data2.groupby('product').title.value_counts(), x='title', y = data2.groupby('product').title.value_counts().index.get_level_values('title'), 
          color = data2.groupby('product').title.value_counts().index.get_level_values('product'), title = 'Product funnel')

### We should consider behaviour of desktop and moblie users separately

## Mobile users analytics

### We have 14% conversion from show to click, 16.2% conversion from click to order and 2.2% from show to order.

In [ ]:
fig = go.Figure(go.Funnel(x = data2[data2.site_version == 'mobile'].title.value_counts().reset_index()['title'],
                          y = data2[data2.site_version == 'mobile'].title.value_counts().reset_index()['index'],
                           textinfo = "value+percent initial"))
fig.show()

In [ ]:
mobile = data2[data2.site_version == 'mobile']
mobile['date'] = pd.to_datetime(mobile['date'])
mobile

### Check seasonal trends

### This is a line through the all time, there are some repeating ups and downs for banner_click and order (it becomes visible when you look at the lines of each action separately, for it you should click the legend)

In [ ]:
px.line(mobile.groupby('date').title.value_counts(), x = mobile.groupby('date').title.value_counts().index.get_level_values(0),
       y = 'title', color = mobile.groupby('date').title.value_counts().index.get_level_values(1))

In [ ]:
mobile['weekday'] = mobile['date'].dt.weekday
mobile

### For mobile users the highest number of ad showings was between 8 a.m. and 10 a.m. and 4 p.m. and 6 p.m., there is the same statistics for clicks. But orders show another dynamics: the highest number of orders was from 2 p.m. to 5 p.m.

In [ ]:
px.line(pd.DataFrame(mobile.groupby('hour').title.value_counts()), x = pd.DataFrame(mobile.groupby('hour').title.value_counts()).index.get_level_values(0),
        y = pd.DataFrame(mobile.groupby('hour').title.value_counts()).title, color = pd.DataFrame(mobile.groupby('hour').title.value_counts()).index.get_level_values(1),
       title = 'Mobile users actions through the day')

In [ ]:
data[data.site_version == 'mobile'].groupby('product').title.value_counts()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Accessories',
    y = ["banner_show", "banner_click", "order"],
    x = [1030951,117723,22430],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Clothes',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [1035058,187814,45738],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Company',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [ 1102861,116357, 0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Sneakers',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [1041430,161177,35154],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Sports_nutrition',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [1048375,131048,12219],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.show()

#### Funnel above shows us:
* #### the most clickable banners for mobile users are with clothes, also these banners have the highest conversion from click to order (24.4%). It means that clothes has a great demand and banners look attractive and informative (when going to the site, the future client sees the product that he expected to see, with the necessary characteristics);
* #### banners with sneakers are quite worse than clothes, but although they are clickable and have a 2.8% conversion to order (from click to order 21.8%);
* #### banners with ports_nutrition have macroconversion 13%, but from click to order it is only 9.3%. Possible problems:
    * #### there is broken links from banner;
    * #### banner attract not target audience (when people go to the site they see not what they want, not what they expect from banner);
    * #### there is not enough informantion about goods on the site, so client stay 'cold' and leave;
    * #### conpetitors give much more cooler than ours (price, quality, assortment etc.)
* #### banners with accessories are not so clickable (11%), but have a good conversion from the click to order (19.1%). We can make a banner more attractive and increase sales of accessories.
* #### banners about company are not so clickable for mobile users.

In [ ]:
clicks = mobile[mobile.user_id.isin(mobile[mobile.title == 'banner_click'].user_id.unique())]
clicks.sort_values(['user_id', 'date'])
orders = mobile[mobile.user_id.isin(mobile[mobile.title == 'order'].user_id.unique())]
orders.sort_values(['user_id', 'date'])

In [ ]:
orders['step'] = orders['product'].astype('object')+'_'+orders['title'].astype('object')
orders = orders.sort_values(['user_id', 'date', 'hour'])

In [ ]:
px.scatter(orders.groupby(['hour', 'weekday']).step.value_counts(), x = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(0),
           y = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(1), size = 'step', 
           color = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(2), title = 'Lead actions')
        

#### From chart above I got an information (click the legend to get actions statistics separately:
* #### all of banners were shown at about the same time
* #### the most clickable time for clothes banners is 7-9 a.m., but orders are made throughout the day: from 10 a.m. to midnight;
* #### for sneakers it is the same;
* #### there is no so much clicks for accessories banners, the highest number of it is 7-9 a.m. too., the majority of accessorires are ordered at the daytime (11 a.m.- 8 p.m.)
* #### there is no so much clicks for sport nutrion banners, the highest number of it is 7-9 a.m. too., the majority of sport nutrition are ordered at the daytime (11 a.m.- 8 p.m.)


In [ ]:
a = orders.groupby('user_id').step.unique().reset_index()
a = a.merge(pd.DataFrame(a['step'].values.tolist()), left_on = pd.DataFrame(a['step'].values.tolist()).index, right_on = a.index)
a = a.drop(['key_0','step'], axis = 1)
a = a.fillna('No action')

In [ ]:
px.parallel_categories(a, dimensions = a.columns[1:], title = 'Full customer map journey')

### Check what banners made clients order something after first banner seeing.

In [ ]:
after_1 = a[(a[1] == 'clothes_order')|(a[1] == 'accessories_order')|(a[1] == 'sports_nutrition_order')|(a[1] == 'sneakers_order')]
px.parallel_categories(after_1, dimensions = after_1.columns[1:])

### As we can see, the majority of first orders (lot's of them are clothes order, then sneakers and accessories) were made after company banner seeing. After this order the majority of people didn't do anything.

### Check what banners were clicked and then orders were made

In [ ]:
after_1 = a[(((a[1] == 'clothes_banner_click')|(a[1] == 'accessories_banner_click')|(a[1] == 'sports_nutrition_banner_click')|(a[1] == 'sneakers_banner_click')|(a[1] == 'company_banner_click'))&
             ((a[2] == 'clothes_order')|(a[2] == 'accessories_order')|(a[2] == 'sports_nutrition_order')|(a[2] == 'sneakers_order')))]
px.parallel_categories(after_1, dimensions = after_1.columns[1:])

In [ ]:
a[(((a[1] == 'clothes_banner_click')|(a[1] == 'accessories_banner_click')|(a[1] == 'sports_nutrition_banner_click')|(a[1] == 'sneakers_banner_click')|(a[1] == 'company_banner_click'))&
             ((a[2] == 'clothes_order')|(a[2] == 'accessories_order')|(a[2] == 'sports_nutrition_order')|(a[2] == 'sneakers_order')))].groupby(2)[1].value_counts()

* #### The least clickable banner for accessories orderer is accessories banner (651 clicks) and company banner (689 clicks) and the most clickable is clothes (1145 clicks). Also making a decision to buy accessories were made after clicking the sneakers banner (967 clicks) and sport_nutrition banner (792 clicks).
* #### For clothes orders the most clickable baneer is with sneakers (2164 clicks). Slightly less conversion to buying clothing from banners with clothes (1697 clicks), sports nutrition (1692 clicks), accessories (1506 clicks) and company (1493 clicks).
* #### For sneakers orders the most clickable banner is with clothes (1841 clicks). Slightly less conversion to buying sneakers from banners with sports nutrition (1234 clicks), sneakers (1225 clicks), company (1215 clicks) and accessories (1138 clicks).
* #### For sport nutrition orders the most clickable banner is clothes (632 clicks). Slightly less conversion to buying sport nutrition from sneakers banner (504 clicks), accessories banners (401 clicks), sport nutrons banner (384 banners) and company banner (374 clicks)

### Check what was going on with people who made first orders after first banner seeing.

In [ ]:
after_1_2 = a[((a[1] == 'clothes_order')|(a[1] == 'accessories_order')|(a[1] == 'sports_nutrition_order')|(a[1] == 'sneakers_order'))
              &((a[3] == 'clothes_banner_click')|(a[3] == 'accessories_banner_click')|(a[3] == 'sports_nutrition_banner_click')|(a[3] == 'sneakers_banner_click')|
               (a[3] == 'clothes_order')|(a[3] == 'accessories_order')|(a[3] == 'sports_nutrition_order')|(a[3] == 'sneakers_order'))]
px.parallel_categories(after_1_2, dimensions = after_1_2.columns[3:])

### They saw different banners, clicked them, but then there was no action anymore. There is only 13 new orders of accessorires after sneakers, nutrion and company banner show and 17 clothes orders after accessories, sneakers and company banner show.

### Check clicking dynamics for mobile users

In [ ]:
px.line(mobile[mobile.title == 'banner_click'].groupby('date')['product'].value_counts(), 
        x = mobile[mobile.title == 'banner_click'].groupby('date')['product'].value_counts().index.get_level_values(0),
        y = 'product',
        color = mobile[mobile.title == 'banner_click'].groupby('date')['product'].value_counts().index.get_level_values(1), title = 'Microconversion from mobile users dynamics')

### There is a sharp increasing of clicking on company banners from January, 20 to February, 17. Then it started changing in the same way with onther banners. Also there is regular increasings and decreasing for each product (it is seasonal weekly trend (in the start of week there is a increase, in the middle - a decrease, then on thursday we have an increase again, and a decrease on holidays again.

### Check macroconversion dynamics for mobile users

In [ ]:
px.line(mobile[mobile.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts(), 
        x = mobile[mobile.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(0),
        y = 'product',
        color = mobile[mobile.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(2), 
        hover_data = [mobile[mobile.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(1)], 
        title = 'Macroconversion dynamics from mobile users')

In [ ]:
px.bar(pd.DataFrame(mobile.groupby('weekday').title.value_counts()), x = pd.DataFrame(mobile.groupby('weekday').title.value_counts()).index.get_level_values(0),
       y = pd.DataFrame(mobile.groupby('weekday').title.value_counts()).title, 
      color = pd.DataFrame(mobile.groupby('weekday').title.value_counts()).index.get_level_values(1))

### There is the same trend for macroconversion of mobile users as for microconversion

## Desktop users analytics

In [ ]:
desktop = data2[data2.site_version == 'desktop']
desktop['date'] = pd.to_datetime(desktop['date'])
desktop

### This funnel shows that desktop banners have very low clickability, but a very high conversion from click to order. It means that it attracts only 'hot' leads who are ready to buy and who find in assortment exatly what they need. But for 'warm' and 'cold' leads this banner is not so good.

In [ ]:
fig = go.Figure(go.Funnel(x = data2[data2.site_version == 'desktop'].title.value_counts().reset_index()['title'],
                          y = data2[data2.site_version == 'desktop'].title.value_counts().reset_index()['index'],
                           textinfo = "value+percent initial"))
fig.show()

### On the chart below we can see that in the start of marketing company clickicking and ordering moved in opposite directions: clicking decreased, but ordering decreased, after 3 months they started moving in the same directions, but ordering line was higher. 

In [ ]:
px.line(desktop.groupby('date').title.value_counts(), x = desktop.groupby('date').title.value_counts().index.get_level_values(0),
       y = 'title', color = desktop.groupby('date').title.value_counts().index.get_level_values(1), title = 'Desktop users activity')

### For desktop users the highest number of ad showings was between 8 a.m. and 10 a.m. and 4 p.m. and 6 p.m., there is the same statistics for clicks. But orders show another dynamics: the highest number of orders was from 2 p.m. to 5 p.m.

In [ ]:
px.line(pd.DataFrame(desktop.groupby('hour').title.value_counts()), x = pd.DataFrame(desktop.groupby('hour').title.value_counts()).index.get_level_values(0),
        y = pd.DataFrame(desktop.groupby('hour').title.value_counts()).title, color = pd.DataFrame(desktop.groupby('hour').title.value_counts()).index.get_level_values(1),
       title = 'Desktop users actions through the day')

In [ ]:
data2[data2.site_version == 'desktop'].groupby('product').title.value_counts()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Accessories',
    y = ["banner_show", "banner_click", "order"],
    x = [410003, 18531, 22121],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Clothes',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [418070, 32781, 66977],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Company',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [477374,28464,0],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Sneakers',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [ 411597, 21419, 32565],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Sports_nutrition',
    y = ["banner_show", "banner_click", "order"],
    orientation = "h",
    x = [417595,13870,11518],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.show()

### Funnel above shows that all desktop banners have a lov clickability for all products, but we have a good conversion from click to order. It means that all clients who saw banners made several orders (it does not refer to company banners as it has not conversion to order)

In [ ]:
desktop['weekday'] = desktop['date'].dt.weekday
desktop

In [ ]:
clicks_desk = desktop[desktop.user_id.isin(desktop[desktop.title == 'banner_click'].user_id.unique())]
clicks_desk.sort_values(['user_id', 'date'])
orders_desk = desktop[desktop.user_id.isin(desktop[desktop.title == 'order'].user_id.unique())]
orders_desk.sort_values(['user_id', 'date'])

In [ ]:
orders_desk['step'] = orders_desk['product'].astype('object')+'_'+orders_desk['title'].astype('object')
orders_desk = orders_desk.sort_values(['user_id', 'date', 'hour'])

In [ ]:
px.scatter(orders_desk.groupby(['hour', 'weekday']).step.value_counts(), x = orders_desk.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(0),
           y = orders_desk.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(1), size = 'step', 
           color = orders_desk.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(2), title = 'Lead actions')

#### From chart above I got an information (click the legend to get actions statistics separately:
* #### all of banners were shown at about the same time
* #### the most clickable time for clothes banners is 7-9 a.m., but orders are made throughout the day: from 10 a.m. to 11 p.m.;
* #### for sneakers it is the same;
* #### there is no so much clicks for accessories banners, the highest number of it is 7-9 a.m. too., the majority of accessorires are ordered at the daytime (11 a.m.- 8 p.m.)
* #### there is no so much clicks for sport nutrion banners, the highest number of it is 7-9 a.m. too., the majority of sport nutrition are ordered at the daytime (11 a.m.- 8 p.m.)


In [ ]:
px.bar(desktop[desktop.title == 'banner_click']['product'].value_counts(), title = 'Microconversion of desktop users')

### There is the highest amount of clicks of desktop users on clothes banners (32K), then company banners (28.5K), sneakers (21.5K), accessories (18.5K) and the least amount on sports nutrition banners (13K)

In [ ]:
px.line(desktop[desktop.title == 'banner_click'].groupby('date')['product'].value_counts(), 
        x = desktop[desktop.title == 'banner_click'].groupby('date')['product'].value_counts().index.get_level_values(0),
        y = 'product',
        color = desktop[desktop.title == 'banner_click'].groupby('date')['product'].value_counts().index.get_level_values(1), title = 'Microconversion from desktop users')

### The chart above shows that from january, 20 to March, 3 there were increasing of clicking a company banner and desreasing of clicking all other banners, but from middle of March they started mving in the same directions.

### Check macroconversion dynamics from the destop users

In [ ]:
px.line(desktop[desktop.title == 'order'].groupby('date')['product'].value_counts(), 
        x = desktop[desktop.title == 'order'].groupby('date')['product'].value_counts().index.get_level_values(0),
        y = 'product',
        color = desktop[desktop.title == 'order'].groupby('date')['product'].value_counts().index.get_level_values(1), title = 'Macroconversion from mobile users')

### We can see that clothes macroconversion is much higher, than others. Also there is regular increasings and decreasings for each product.

In [ ]:
desktop[desktop.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts()

In [ ]:
px.line(desktop[desktop.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts(), 
        x = desktop[desktop.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(0),
        y = 'product',
        color = desktop[desktop.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(2), title = 'Macroconversion from mobile users',
       hover_data = [desktop[desktop.title == 'order'].groupby(['date', 'weekday'])['product'].value_counts().index.get_level_values(1)])

### These increasings and decreasings are weekly seasonality: there is high amount of orders on Monday, then it decreases to the middle of the week, on 

In [ ]:
clicks = desktop[desktop.user_id.isin(desktop[desktop.title == 'banner_click'].user_id.unique())]
clicks.sort_values(['user_id', 'date'])
orders = desktop[desktop.user_id.isin(desktop[desktop.title == 'order'].user_id.unique())]
orders.sort_values(['user_id', 'date'])

In [ ]:
orders['step'] = orders['product'].astype('object')+'_'+orders['title'].astype('object')
orders = orders.sort_values(['user_id', 'date', 'hour'])

In [ ]:
px.scatter(orders.groupby(['hour', 'weekday']).step.value_counts(), x = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(0),
           y = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(1), size = 'step', 
           color = orders.groupby(['hour', 'weekday']).step.value_counts().index.get_level_values(2), title = 'Lead actions')


#### From chart above I got an information (click the legend to get actions statistics separately:
* #### if we consider amount of clicks and orders, we can see that amount of clicks is much less than amount of orders. It makes me think that pe
* #### all of banners were shown at about the same time (the highest amount of showings was from 7 to 9 a.m. and from 4 to 6 p.m.
* #### the most clickable time for clothes banners is 7-9 a.m., but orders are made throughout the day: from 10 a.m. to midnight;
* #### for sneakers it is the same;
* #### there is no so much clicks for accessories banners, the highest number of it is 7-9 a.m. too., the majority of accessorires are ordered at the daytime (11 a.m.- 8 p.m.)
* #### there is no so much clicks for sport nutrion banners, the highest number of it is 7-9 a.m. too., the majority of sport nutrition are ordered at the daytime (11 a.m.- 8 p.m.)

In [ ]:
a = orders.groupby('user_id').step.unique().reset_index()
a = a.merge(pd.DataFrame(a['step'].values.tolist()), left_on = pd.DataFrame(a['step'].values.tolist()).index, right_on = a.index)
a = a.drop(['key_0','step'], axis = 1)
a = a.fillna('No action')

In [ ]:
px.parallel_categories(a, dimensions = a.columns[1:], title = 'Full customer map journey')

In [ ]:
after_1 = a[(a[1] == 'clothes_order')|(a[1] == 'accessories_order')|(a[1] == 'sports_nutrition_order')|(a[1] == 'sneakers_order')]
px.parallel_categories(after_1, dimensions = after_1.columns[1:])

### From graphics above we can see that majority of first orders are clothes orders, the second one by amount of orders is sneakers. The majority of first orders were made after company banner seeing.

### Check what banners were clicked before first order was made

In [ ]:
after_1 = a[(((a[1] == 'clothes_banner_click')|(a[1] == 'accessories_banner_click')|(a[1] == 'sports_nutrition_banner_click')|(a[1] == 'sneakers_banner_click')|(a[1] == 'company_banner_click'))&
             ((a[2] == 'clothes_order')|(a[2] == 'accessories_order')|(a[2] == 'sports_nutrition_order')|(a[2] == 'sneakers_order')))]
px.parallel_categories(after_1, dimensions = after_1.columns[1:])

In [ ]:
a[(((a[1] == 'clothes_banner_click')|(a[1] == 'accessories_banner_click')|(a[1] == 'sports_nutrition_banner_click')|(a[1] == 'sneakers_banner_click')|(a[1] == 'company_banner_click'))&
             ((a[2] == 'clothes_order')|(a[2] == 'accessories_order')|(a[2] == 'sports_nutrition_order')|(a[2] == 'sneakers_order')))].groupby(2)[1].value_counts()

* #### The majority of clothes orders were made after company banner clicks (2011 clicks) and clothes banner clicks (1133 clicks). Much less conversion from sneakers banner clicks (533 clicks), accessroies banner clicks (511 clicks) and sports nutrition banners (329 clicks)
* #### The majority of sneakers orders were made after clothes banner clicks (556) and company banner clicks (503). Slightly less conversion  from sneakers banners (393), accessorires (332) and sport nutrition banners (243)
* #### The majority of accessories orders were made after clothes banner clicks (405) and company banner clicks (334). Slightly less conversion from sneakers banner clicks (261), accessories banner clicks (257)
* #### The majority of sports nutrition orders were made after clothes banner clicks (203). Less conversion from company banner clicks (161), sneakers banner clisks (123), accessories banner clicks (111), sports nutrition banner clicks (102)

### Check what was going on with people who made first orders after first banner seeing

In [ ]:
after_1_2 = a[((a[1] == 'clothes_order')|(a[1] == 'accessories_order')|(a[1] == 'sports_nutrition_order')|(a[1] == 'sneakers_order'))
              &((a[3] == 'clothes_banner_click')|(a[3] == 'accessories_banner_click')|(a[3] == 'sports_nutrition_banner_click')|(a[3] == 'sneakers_banner_click')|
               (a[3] == 'clothes_order')|(a[3] == 'accessories_order')|(a[3] == 'sports_nutrition_order')|(a[3] == 'sneakers_order'))]
px.parallel_categories(after_1_2, dimensions = after_1_2.columns[3:])

### The majority of them clicked different banners but after that there was no action. But about 160 people made a second purchase (they came from different banners)